# Capítulo 1 - O Problema

Temos no CEFET MG um grupo chamado Grupo de Computação Competitiva (GCC) que tem como uma de suas atividades a participação na maratona de programação.

A maratona de programação é uma competição onde equipes competem entre si em um período de tempo pré determinado a tentar resolver o maior número de problema em menos tempo.

O objetivo deste trabalho, é desenvolver um algoritmo de recomendação que ajude no treinamento dos competidores para a preparação para a maratona de programação.

Atualmente, o treinamento funciona da seguinte maneira. Existem alguns sites, chamados de juízes online, que funcionam como um repositório de problemas. O competidor então consegue ler o problema, desenvolver uma solução e submete-la para o juiz. O juiz então dará a resposta se a solução foi aceita ou não.

O problema do treinamento atual é que existem milhares de problemas a serem resolvidos. Isso faz com que o competidor perca tempo em problemas que não acrescentem em conteúdo para o seu desenvolvimento, ou então, escolha problemas muito difíceis para serem resolvidos, o que torna seu treinamento desmotivador.


# Capítulo 2 - Dataset


Foram coletados do site URI Online Judge, mais de 200 mil soluções de mais de 600 usuários. Cada solução é composta por:

 - **user_id**: usuário que resolveu o problema
 - **problem_id**: problema que foi resolvido
 - **date**: data de quando o problema foi resolvido
 - **category_id**: categoria do problema resolvido
 - **level**: dificuldade do problema resolvido
 - **solved**: quantidade total de usuários do juiz que resolveram o problema
 - **name**: nome do problema
 - **topics**: assuntos relacionados ao conteúdo do problema

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

def datefunc(date):
    d = datetime.strptime(date,'%m/%d/%y, %I:%M:%S %p')
    return d.strftime('%y/%m/%d %I:%M')

solutions_df = pd.read_csv('solutions.csv', delimiter='\t', converters={'date': datefunc})
solutions_df[:5]

,user_id,problem_id,date,category_id,level,solved,name,topics
0,40980,1001,15/02/25 06:15,1,1,127331,Extremely Basic,sequential
1,40980,1002,15/02/25 07:10,1,1,87459,Area of a Circle,sequential
2,40980,1003,15/02/25 06:17,1,1,89856,Simple Sum,sequential
3,40980,1004,15/02/25 06:19,1,1,85938,Simple Product,sequential
4,40980,1005,15/02/25 07:31,1,1,73427,Average 1,sequential


Quantidade de soluções do dataset:

In [2]:
len(solutions_df)

218619

Quantidade de problemas do dataset:

In [3]:
solutions_df['problem_id'].nunique()

1461

Quantidade de usuários do dataset:

In [4]:
solutions_df['user_id'].nunique()

618

Quantidade de categorias do dataset:

In [5]:
solutions_df['category_id'].nunique()

8

# Capítulo 3 - Tratamento dos dados

Ainda não é necessário.

# Capítulo 4 - Desenvolvimento

Uma série de técnicas de Machine Learning serão utilizadas a fim de obter o resultado final. Serão aqui descritas passo a passo, os procedimentos adotados.

obs: cada exemplo poderia conter o link de um problema

- storytelling

Em todo período acadêmico, o conteúdo aprendido é desenvolvido de forma gradual. Como as aulas de cálculo, onde temos o estudo de funções, limites, derivadas e integrais. Todo o conteúdo aprendido inicialmente é usado como base para os demais.

Da mesma forma funciona o treinamento para a maratona de programação. Primeiramente temos os problemas que envolvem estruturas de dados, como listas encadeadas. Posteriormente lidamos com árvores, grafos e etc.

Com isso podemos supor que todos os problemas resolvidos contribuem com o aprendizado do competidor em algum tópico. E ai está o nosso primeiro problema: **Identificar os tópicos necessários a se saber para a solução de um problema**

Tendo isso em mente, essa será a primeira tarefa desenvolvida. Para isso faremos o seguinte.

Seja p um problema e u um usuario do nosso dataset. Serão filtradas todas as soluções submetidas pelo usuário u antes da solução do problema p. Isso nos permitirá identificar quais os tópicos que foram desenvolvidos pelo usuário u que o capacitaram a solucionar o problema p.

Para isso iremos utilizar a técnica de Collaborative Filtering.

## Colaborative Filtering

Imagine um dataset composto por avaliações de filmes, onde cada usuário avalia um filme em uma nota entre 1 e 5. Teriamos uma tabela da seguinte forma:

| Filmes  | Ana | Beth | Carlos | Davi |
|---------|:---:|:----:|:------:|:----:|
| Querido John |  5  |   5  |    0   |   0  |
| Titanic |  5  |   5  |    0   |   0  |
| Batman |  0  |   0  |    5   |   5  |
| Thor |  0  |   0  |    5   |   5  |

Baseado na avaliação dos usuários podemos encontrar o gênero de cada filme.

| Filmes  | Ana | Beth | Carlos | Davi | Ação | Romance|
|---------|:--:|:--:|:--:|:--:|:--:|:--:|
| Querido John | 5 | 5 | 0 | 0 | ? | ? |
| Titanic | 5 | 5 | 0 | 0 | ? | ? |
| Batman | 0 | 0 | 5 | 5 | ? | ? |
| Thor | 0 | 0 | 5 | 5 | ? | ? |

Para que isso seja possível, existe um vetor de preferênca de cada usuário $\theta$ que indica o quanto cada usuário gosta de cada gênero de filme. E cada filme tem um vetor $X$ corresponde a quanto o filme pertence ao gênero indicado.

Podemos então estimar um valor de $\theta$ dos usuários a fim de descobrir o valor de $X$. Daí fazemos o contrário. É quase uma questão de quem nasceu primeiro, o ovo ou a galinha.

Descrever o algoritmo e o exemplo pra provar que funciona. :)

```
Pegar os problemas.
Para cada problema p em problemas:
 Pegar os usuarios que resolveram o problema p
 Para cada usuario
  Pegar os problemas resolvidos antes do problema p
  
 tranformar isso em uma matriz problema x usuario
 rodar o algoritmo
 salvar o resultado
 ```

In [9]:
def get_solution_date(solutions_df, user_id, problem_id):
    solution = solutions_df[(solutions_df['problem_id'] == problem_id) & (solutions_df['user_id'] == user_id)]
    solution_date = solution['date'].values[0]

    return solution_date


def get_past_problems(solutions_df, user_id, problem_id):
    solution_date = get_solution_date(solutions_df, user_id, problem_id)
    past_solutions = solutions_df[(solutions_df['date'] < solution_date) & (solutions_df['user_id'] == user_id)]
    problems = past_solutions['problem_id']

    return list(problems)


def get_users_that_solved_the_problem(solutions_df, problem_id):
    solutions_of_the_problem_df = solutions_df[solutions_df['problem_id'] == problem_id]
    users = solutions_of_the_problem_df['user_id']
    
    return list(users)


def get_problems(solutions_df):
    problems = set(solutions_df['problem_id'])

    return list(problems)


def get_solution_matrix(solutions_df, problem):
    users = get_users_that_solved_the_problem(solutions_df, problem)

    solutions = []
    min_problem = 9999
    max_problem = 0

    for user in users:
        past_problems = get_past_problems(solutions_df, user, problem)

        solutions.append(past_problems)

        min_problem = min([min_problem] + past_problems)
        max_problem = max([max_problem] + past_problems)

    nr_problems = max_problem - min_problem + 1
    nr_users = len(users)
    matrix = np.zeros((nr_problems, nr_users))

    for user in range(nr_users):
        for problem in solutions[user]:
            problem_index = problem - min_problem
            try:
                matrix[problem_index][user] = 1 # ou level do problema
            except:
                import pdb; pdb.set_trace()
    return matrix


# http://www.bogotobogo.com/python/python_numpy_batch_gradient_descent_algorithm.php
def gradient_descent(x, y, alpha=0.01, nr_iterations=1000):

    # m - número de amostras
    # n - número features
    m, n = x.shape

    # Insere coluna theta zero = 1
    x = np.c_[ np.ones(m), x]

    theta = np.ones(n + 1)
    x_transpose = x.transpose()

    for iter in range(0, nr_iterations):
        hypothesis = np.dot(x, theta)
        loss = hypothesis - y
        J = np.sum(loss ** 2) / (2 * m)
        gradient = np.dot(x_transpose, loss) / m         
        theta = theta - alpha * gradient

    return theta


NR_FEATURES = 4

def get_theta_of_problem(matrix):
    nr_problems, nr_users = matrix.shape

    theta_users = np.random.rand(nr_users, NR_FEATURES)
    theta_problems = np.zeros((nr_problems, NR_FEATURES))


    # matrix
    #     u1  u2  u3
    # p1   1
    # p2   1  1    1
    # p3      1

    matrix_t = np.transpose(matrix)
    
    print('nr_problems', nr_problems)
    print('nr_users', nr_users)
    
    
    i = 0
    while i < 10:
        i += 1
        print(i)

        for problem in range(nr_problems):
            x = np.array(
                theta_users
            )

            y = np.array(
                matrix[problem]
            )
            
            print('.', end='', flush=True)

            theta_problems[problem] = gradient_descent(x, y)[1:]

        for user in range(nr_users):
            x = np.array(
                theta_problems
            )

            y = np.array(
                matrix_t[user]
            )
            
            print(',', end='', flush=True)

            theta_users[user] = gradient_descent(x, y)[1:]

        print(theta_problems)
    
    return theta_problems

In [ ]:
matrix = get_solution_matrix(solutions_df, 1001)
get_theta_of_problem(matrix)

nr_problems 1491
nr_users 618
1
........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................